<a href="https://colab.research.google.com/github/NilashishDe/Honeywell-Hackathon/blob/main/flightdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

def clean_flight_data(file_path):
    """
    This function takes the path to one of your raw Excel files
    and returns a cleaned, structured DataFrame.
    """
    # Use pd.read_excel() - the 'on_bad_lines' argument has been REMOVED.
    df = pd.read_excel(
        file_path,
        header=None, # Treat the first row as data
        skiprows=1, # Skip the original header row
        names=[
            'S.No', 'Flight Number', 'Date', 'From', 'To', 'Aircraft',
            'Flight time', 'STD', 'ATD', 'STA', 'col10', 'ATA', 'col12', 'col13'
        ]
    )

    # --- Data Cleaning (This part remains the same) ---
    df['Flight Number'].fillna(method='ffill', inplace=True)
    df.dropna(subset=['Date'], inplace=True)
    df.drop(columns=['S.No', 'col10', 'col12', 'col13'], inplace=True)

    def clean_ata(time_str):
        if isinstance(time_str, str) and 'Landed' in time_str:
            time_part = time_str.replace('Landed', '').strip()
            try:
                return pd.to_datetime(time_part, format='%I:%M %p').strftime('%H:%M:%S')
            except (ValueError, TypeError):
                return None
        elif hasattr(time_str, 'strftime'):
             return time_str.strftime('%H:%M:%S')
        return time_str

    df['ATA_cleaned'] = df['ATA'].apply(clean_ata)

    # Ensure time columns are strings before concatenation
    df['STA'] = df['STA'].astype(str)
    df['ATA_cleaned'] = df['ATA_cleaned'].astype(str)

    # Clean the date column to handle potential timestamp objects
    df['Date'] = pd.to_datetime(df['Date']).dt.date.astype(str)

    df['STA_datetime'] = pd.to_datetime(df['Date'] + ' ' + df['STA'], errors='coerce')
    df['ATA_datetime'] = pd.to_datetime(df['Date'] + ' ' + df['ATA_cleaned'], errors='coerce')

    df['delay_minutes'] = (df['ATA_datetime'] - df['STA_datetime']).dt.total_seconds() / 60
    df['is_delayed'] = df['delay_minutes'].apply(lambda x: 1 if x > 15 else 0)
    df['day_of_week'] = df['STA_datetime'].dt.day_name()
    df['scheduled_hour'] = df['STA_datetime'].dt.hour

    final_df = df[[
        'Flight Number', 'Date', 'From', 'To', 'Aircraft', 'STD', 'STA', 'delay_minutes',
        'is_delayed', 'day_of_week', 'scheduled_hour'
    ]].copy()
    final_df.dropna(subset=['delay_minutes', 'To', 'Aircraft'], inplace=True)

    print(f"Cleaned {file_path}. Found {len(final_df)} valid flights.")
    return final_df

# --- Main Execution ---
if __name__ == '__main__':
    # Your Excel filenames
    file1 = '6-9flight.xlsx'
    file2 = '9-12flight.xlsx'

    # Clean both Excel files
    df1 = clean_flight_data(file1)
    df2 = clean_flight_data(file2)

    # Combine them into one big dataset
    combined_df = pd.concat([df1, df2], ignore_index=True)

    # Save the final, clean dataset to a CSV file
    combined_df.to_csv('cleaned_flight_data.csv', index=False)

    print("\nSuccessfully combined and saved cleaned data to 'cleaned_flight_data.csv'")
    print(f"Total valid flights in combined dataset: {len(combined_df)}")
    print(combined_df.head())

Cleaned 6-9flight.xlsx. Found 384 valid flights.
Cleaned 9-12flight.xlsx. Found 391 valid flights.

Successfully combined and saved cleaned data to 'cleaned_flight_data.csv'
Total valid flights in combined dataset: 775
  Flight Number        Date          From                To       Aircraft  \
0                2025-07-25  Mumbai (BOM)  Chandigarh (IXC)  A20N (VT-EXU)   
1                2025-07-24  Mumbai (BOM)  Chandigarh (IXC)  A20N (VT-RTJ)   
2                2025-07-23  Mumbai (BOM)  Chandigarh (IXC)  A20N (VT-TQB)   
3                2025-07-22  Mumbai (BOM)  Chandigarh (IXC)  A20N (VT-RTU)   
4                2025-07-21  Mumbai (BOM)  Chandigarh (IXC)  A20N (VT-EXK)   

        STD       STA  delay_minutes  is_delayed day_of_week  scheduled_hour  
0  06:00:00  08:10:00            4.0           0      Friday               8  
1  06:00:00  08:10:00           -9.0           0    Thursday               8  
2  06:00:00  08:10:00           13.0           0   Wednesday               

/tmp/ipython-input-4123803572.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Flight Number'].fillna(method='ffill', inplace=True)
/tmp/ipython-input-4123803572.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Flight Number'].fillna(method='ffill', inplace=True)
/tmp/ipython-input-4123803572.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace meth

In [6]:
import pandas as pd

# Load your cleaned dataset
df_clean = pd.read_csv('cleaned_flight_data.csv')

# Check for missing values in each column and sum them up
missing_values = df_clean.isnull().sum()

print("Missing values in each column after cleaning:")
print(missing_values)

Missing values in each column after cleaning:
Flight Number     0
Date              0
From              0
To                0
Aircraft          0
STD               0
STA               0
delay_minutes     0
is_delayed        0
day_of_week       0
scheduled_hour    0
dtype: int64


In [12]:
# Final Script: Comparing 5 Models (Logistic Regression, Decision Tree, Random Forest, Gradient Boosting, XGBoost)
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import warnings

# Import all the models we want to compare
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

warnings.filterwarnings('ignore')

# 1. Load the clean data
df = pd.read_csv('cleaned_flight_data.csv')

# 2. Define Features (X) and Target (y)
features = ['From', 'To', 'Aircraft', 'day_of_week', 'scheduled_hour']
target = 'is_delayed'
X = df[features]
y = df[target]

# 3. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Set up the preprocessing pipeline for categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'),
         ['From', 'To', 'Aircraft', 'day_of_week'])
    ],
    remainder='passthrough'
)

# 5. Define the models to be evaluated, exactly as per your list
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# 6. Loop to train, evaluate, and find the best model using cross-validation
results = {}
best_model_name = ""
best_model_score = 0.0

print("--- Comparing Models using 5-Fold Cross-Validation ---")
for name, model in models.items():
    # Create the full pipeline: Preprocessing -> Model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])

    # Use k-fold cross-validation to get a stable performance score
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    mean_score = cv_scores.mean()
    results[name] = mean_score

    print(f"{name} - Cross-Validation Accuracy: {mean_score:.4f}")

    # Keep track of the best model
    if mean_score > best_model_score:
        best_model_score = mean_score
        best_model_name = name

print(f"\n🏆 Best performing model is: {best_model_name} with a CV score of {best_model_score:.4f}")

# 7. Train the best model on the entire training set and save it
best_model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('classifier', models[best_model_name])])
best_model_pipeline.fit(X_train, y_train)

# 8. Final evaluation on the unseen test data
predictions = best_model_pipeline.predict(X_test)
print("\n--- Final Report for the Best Model on Unseen Test Data ---")
print(classification_report(y_test, predictions))

# 9. Save the champion model
joblib.dump(best_model_pipeline, 'flight_delay_model.joblib')
print(f"✅ Successfully saved the winning '{best_model_name}' model to 'flight_delay_model.joblib'")

--- Comparing Models using 5-Fold Cross-Validation ---
Logistic Regression - Cross-Validation Accuracy: 0.8645
Decision Tree - Cross-Validation Accuracy: 0.8306
Random Forest - Cross-Validation Accuracy: 0.8806
Gradient Boosting - Cross-Validation Accuracy: 0.8629
XGBoost - Cross-Validation Accuracy: 0.8790

🏆 Best performing model is: Random Forest with a CV score of 0.8806

--- Final Report for the Best Model on Unseen Test Data ---
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       123
           1       0.90      0.59      0.72        32

    accuracy                           0.90       155
   macro avg       0.90      0.79      0.83       155
weighted avg       0.90      0.90      0.90       155

✅ Successfully saved the winning 'Random Forest' model to 'flight_delay_model.joblib'
